In [3]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta


SEARCH_WORD = "EXA FIRST"

DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"

spreadSheet_ids = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "アスカ狭山店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    }

# 検索キーワードよりホール名取得
SPREADSHEET_ID = spreadSheet_ids[SEARCH_WORD]

# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)

# # Table name 取得
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cursor.fetchall()
# print(tables)

cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + SEARCH_WORD + "%",)
)
results = cursor.fetchall()

# 結果表示
if results:
    print(f"🔍 '{SEARCH_WORD}' を含むホール名の検索結果:")
    for hall_id, hall_name in results:
        print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
else:
    print(f"❌ '{SEARCH_WORD}' を含むホール名は見つかりませんでした。")

query = """
-- 出玉データにホール名と機種名を結合して取得
SELECT
    r.*, 
    h.name AS hall_name,     -- ホール名を追加
    m.name AS model_name     -- 機種名を追加
FROM results r
JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
JOIN models m ON r.model_id = m.model_id  -- 機種と結合
WHERE h.name = ?  -- 指定ホールのみ
AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
ORDER BY r.date DESC, r.unit_no ASC;
"""

# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals):
    bb_medals = 239.25
    rb_medals = 95.25
    cherry_rate_high = 0.04228
    cherry_rate_low = 0.05847
    replay_rate = 0.411
    denominator_inner = (-medals - (game*3 - (bb*bb_medals + rb*rb_medals + game*replay_rate + game*cherry_rate_high))) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    
    return grape_rate

df = pd.read_sql_query(query, conn, params=(hall_name,))
conn.close()

df["date"] = pd.to_datetime(df["date"])
df.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
df = df[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
df["BB_rate"] = (df["game"] / df["BB"]).round(1)
df["RB_rate"] = (df["game"] / df["RB"]).round(1)
df["grape_rate"] = grape_calculator_myfive(df["game"], df["BB"], df["RB"], df["medals"]).round(2)
df["Total_rate"] = (df["game"] / (df["BB"] + df["RB"])).round(1)
df["month"] = df["date"].dt.strftime("%Y-%m")
df["day"] = df["date"].dt.day
df["weekday"] = df["date"].dt.weekday
df["unit_last"] = df["unit_no"].astype(str).str[-1]

df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

print(df.shape)
df.head()
df["model_name"].unique()

🔍 'EXA FIRST' を含むホール名の検索結果:
 - hall_name: EXA FIRST, hall_id: 108
(57456, 16)


array(['ゴーゴージャグラー3', 'マイジャグラーV', 'ファンキージャグラー2', 'ミスタージャグラー',
       'アイムジャグラーEX-TP', 'ハッピージャグラーVIII', 'ウルトラミラクルジャグラー', 'ジャグラーガールズ',
       'SミスタージャグラーKK'], dtype=object)

## RB_RATE

In [24]:
model_name = "アイムジャグラーEX-TP"
df_tmp = df[(df["model_name"] == model_name)].copy()
df_tmp = df_tmp[(df_tmp["date"].dt.date <= datetime.date(2025, 4, 17)) & (df_tmp["date"].dt.date >= datetime.date(2025, 1, 1))]

game = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="game", aggfunc="sum", margins=True, margins_name="TOTAL")
bb = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="BB", aggfunc="sum", margins=True, margins_name="TOTAL")
rb = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="RB", aggfunc="sum", margins=True, margins_name="TOTAL")
medals = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="medals", aggfunc="sum", margins=True, margins_name="TOTAL")


In [25]:
rb_rate = (game / rb).round(1)
rb_rate.to_csv(f"{model_name}_rb_rate.csv", encoding="utf_8_sig")
rb_rate.head()

date                   2025-01-01 00:00:00  2025-01-02 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-01-03 00:00:00  2025-01-04 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-01-05 00:00:00  2025-01-06 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-01-07 00:00:00  2025-01-08 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-01-09 00:00:00  2025-01-10 00:00:00  ...  \
model_name    unit_no                                            ...   
アイムジャグラーEX-TP 1100                     NaN                  NaN  ...   
              1101                     NaN                  NaN  ...   
              1102                     NaN                  NaN  ...   
              1103                     NaN                  NaN  ...   
              1104                     NaN                  NaN  ...   

date                   2025-04-08 00:00:00  2025-04-09 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-04-10 00:00:00  2025-04-11 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-04-12 00:00:00  2025-04-13 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-04-14 00:00:00  2025-04-15 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                   629.3                294.4   
    

In [26]:
total_rate = (game / (bb+rb)).round(1)
total_rate.to_csv(f"{model_name}_total_rate.csv", encoding="utf_8_sig")
total_rate.head()

date                   2025-01-01 00:00:00  2025-01-02 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-01-03 00:00:00  2025-01-04 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-01-05 00:00:00  2025-01-06 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-01-07 00:00:00  2025-01-08 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-01-09 00:00:00  2025-01-10 00:00:00  ...  \
model_name    unit_no                                            ...   
アイムジャグラーEX-TP 1100                     NaN                  NaN  ...   
              1101                     NaN                  NaN  ...   
              1102                     NaN                  NaN  ...   
              1103                     NaN                  NaN  ...   
              1104                     NaN                  NaN  ...   

date                   2025-04-08 00:00:00  2025-04-09 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-04-10 00:00:00  2025-04-11 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-04-12 00:00:00  2025-04-13 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-04-14 00:00:00  2025-04-15 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                   209.8                126.2   
    

In [27]:
game.to_csv(f"{model_name}_game.csv", encoding="utf_8_sig")
game.head()

date                   2025-01-01 00:00:00  2025-01-02 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-01-03 00:00:00  2025-01-04 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-01-05 00:00:00  2025-01-06 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-01-07 00:00:00  2025-01-08 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-01-09 00:00:00  2025-01-10 00:00:00  ...  \
model_name    unit_no                                            ...   
アイムジャグラーEX-TP 1100                     NaN                  NaN  ...   
              1101                     NaN                  NaN  ...   
              1102                     NaN                  NaN  ...   
              1103                     NaN                  NaN  ...   
              1104                     NaN                  NaN  ...   

date                   2025-04-08 00:00:00  2025-04-09 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-04-10 00:00:00  2025-04-11 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-04-12 00:00:00  2025-04-13 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                     NaN                  NaN   
              1101                     NaN                  NaN   
              1102                     NaN                  NaN   
              1103                     NaN                  NaN   
              1104                     NaN                  NaN   

date                   2025-04-14 00:00:00  2025-04-15 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1100                  3776.0               7065.0   
    

In [28]:
medals.to_csv(f"{model_name}_medals.csv", encoding="utf_8_sig")
medals.tail()

date                   2025-01-01 00:00:00  2025-01-02 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1152                  -724.0               1276.0   
              1153                  2374.0               -350.0   
              1154                 -1347.0               1579.0   
              1155                  1371.0               -803.0   
TOTAL                               3933.0               -963.0   

date                   2025-01-03 00:00:00  2025-01-04 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1152                   238.0                229.0   
              1153                    15.0                476.0   
              1154                  -779.0              -1312.0   
              1155                   168.0                806.0   
TOTAL                               -793.0               -338.0   

date                   2025-01-05 00:00:00  2025-01-06 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1152                  -644.0               -356.0   
              1153                 -1171.0              -1265.0   
              1154                  1300.0               -485.0   
              1155                   221.0                288.0   
TOTAL                               2777.0              -3929.0   

date                   2025-01-07 00:00:00  2025-01-08 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1152                  -538.0               -868.0   
              1153                  -468.0              -1124.0   
              1154                  1112.0               1382.0   
              1155                   -62.0               -921.0   
TOTAL                              -4644.0              -5759.0   

date                   2025-01-09 00:00:00  2025-01-10 00:00:00  ...  \
model_name    unit_no                                            ...   
アイムジャグラーEX-TP 1152                  1253.0                524.0  ...   
              1153                    -3.0                641.0  ...   
              1154                  -674.0                 50.0  ...   
              1155                  -468.0               -926.0  ...   
TOTAL                                -19.0               3798.0  ...   

date                   2025-04-08 00:00:00  2025-04-09 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1152                  1279.0               -656.0   
              1153                   -74.0               1759.0   
              1154                  -297.0               -185.0   
              1155                  -462.0                 26.0   
TOTAL                                217.0                -69.0   

date                   2025-04-10 00:00:00  2025-04-11 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1152                   -53.0               -724.0   
              1153                  -232.0               -391.0   
              1154                  -485.0               6650.0   
              1155                  1171.0               -629.0   
TOTAL                                322.0               2220.0   

date                   2025-04-12 00:00:00  2025-04-13 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1152                   994.0               -865.0   
              1153                  -444.0               1253.0   
              1154                   521.0               -332.0   
              1155                  2691.0                 76.0   
TOTAL                                413.0              -3496.0   

date                   2025-04-14 00:00:00  2025-04-15 00:00:00  \
model_name    unit_no                                             
アイムジャグラーEX-TP 1152                  1588.0              -1006.0   
    

In [22]:
cumulative_df = medals.iloc[:, ::-1].cumsum(axis=1).iloc[:, ::-1]
cumulative_df = medals.cumsum(axis=1)
cumulative_df.to_csv(f"{model_name}_cumulative_df.csv", encoding="utf_8_sig")
cumulative_df.tail()

date                   2025-01-01  2025-01-02  2025-01-03  2025-01-04  \
model_name    unit_no                                                   
アイムジャグラーEX-TP 1151         -756.0      -859.0       217.0       385.0   
              1152         -724.0       552.0       790.0      1019.0   
              1153         2374.0      2024.0      2039.0      2515.0   
              1154        -1347.0       232.0      -547.0     -1859.0   
              1155         1371.0       568.0       736.0      1542.0   

date                   2025-01-05  2025-01-06  2025-01-07  2025-01-08  \
model_name    unit_no                                                   
アイムジャグラーEX-TP 1151         -491.0     -2132.0     -2770.0     -3649.0   
              1152          375.0        19.0      -519.0     -1387.0   
              1153         1344.0        79.0      -389.0     -1513.0   
              1154         -559.0     -1044.0        68.0      1450.0   
              1155         1763.0      2051.0      1989.0      1068.0   

date                   2025-01-09  2025-01-10  ...  2025-04-07  2025-04-08  \
model_name    unit_no                          ...                           
アイムジャグラーEX-TP 1151        -3161.0     -2787.0  ...    -19131.0    -18231.0   
              1152         -134.0       390.0  ...     -6091.0     -4812.0   
              1153        -1516.0      -875.0  ...      -796.0      -870.0   
              1154          776.0       826.0  ...     -2139.0     -2436.0   
              1155          600.0      -326.0  ...     -2191.0     -2653.0   

date                   2025-04-09  2025-04-10  2025-04-11  2025-04-12  \
model_name    unit_no                                                   
アイムジャグラーEX-TP 1151       -18793.0    -19072.0    -20443.0    -20922.0   
              1152        -5468.0     -5521.0     -6245.0     -5251.0   
              1153          889.0       657.0       266.0      -178.0   
              1154        -2621.0     -3106.0      3544.0      4065.0   
              1155        -2627.0     -1456.0     -2085.0       606.0   

date                   2025-04-13  2025-04-14  2025-04-15  2025-04-16  
model_name    unit_no                                                  
アイムジャグラーEX-TP 1151       -20869.0    -20534.0    -19631.0    -20790.0  
              1152        -6116.0     -4528.0     -5534.0     -5778.0  
              1153         1075.0       413.0       657.0      -275.0  
              1154         3733.0      5257.0      4816.0      4419.0  
              1155          682.0       888.0       -38.0      -147.0  

[5 rows x 104 columns]